**We use Pegasus financial summariser model from the transformers library in order to perform summarisation on the text obtained from 8K forms**

In [10]:
!pip install transformers
!pip install -q sentencepiece

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import json
import pandas as pd

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [13]:
tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus").to(device)

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive  #This is to make a short form for the mydrive location 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [15]:
def findSummary(sentence):
    batch = tokenizer(sentence, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text[0]

In [16]:
def findSummaries(json_file_name,lb,ub):
  f = open(f'/content/{json_file_name}')
  data = json.load(f)
  companies = pd.read_csv("/content/GoodCom.csv")
  ciks = companies["CIK"].astype(int).tolist()
  years = [2021,2020]
  for cik in ciks[lb:ub]:
    for year in years:
      try:
        sentences = data[f"{cik}"][f"{year} Sentences"]
        fhand = open(f'/mydrive/SEC_8K/{cik}_{year}.txt','w+')
        for sentence_object in sentences:
          sentence = sentence_object['sentence']
          date = sentence_object['date']

          summary = findSummary(sentence)
          summary = summary.capitalize()

          random = 'iitidigitalalpha'
          
          fhand.write(summary+random+date+'\n')
        fhand.close()
      except Exception as e:
        print(e)
        continue
  print("Done")

In [17]:
json_file_name = "8k_text_data_mihir_1.json" #name of json file obtained from bulk scraping
lb = 0 #start of range
ub = 3 #end of range

#Create a folder called "SEC_8K" in My Drive section of Google Drive
#Upload json file obtained from scraping in the Files section of Google Collab
#Upload GoodCom.csv in the Files section of Google Collab

findSummaries(json_file_name,lb,ub)

Done
